# Generalized Camera Class for OpenCV
A common problem is that as you move your OpenCv application between
hardware environments or camera technologies, your code breaks.
And in general, its hard to add support for yet another camera.
For example, the Raspberry Pi is fairly complex
because it uses a background capture thread to talk to the camera hardware.
USB based cameras appear as a device, as in `/dev/video*` and
laptop embedded mat have yet another access method.

In [1]:
import cv2
from importlib import import_module
import base64

def send_a_frame():
    capture = cv2.VideoCapture("/home/jeff/Videos/People-Walking-Shot-From-Above.mp4")
    frame = capture.read()[1]
    cnt = cv2.imencode('.png',frame)[1]
    b64 = base64.encodestring(cnt)
    b64 = base64.encodebytes(cnt)
    html = "<html><img src='data:image/png;base64,"+b64 +"'></html"
    send(html)
    
if __name__ == '__main__':
    send_a_frame()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: encodestring() is a deprecated alias, use encodebytes()
  if __name__ == '__main__':


TypeError: expected 1-D data, not 2-D data from ndarray

## `base_camera.py`

In [2]:
import time
import threading
try:
    from greenlet import getcurrent as get_ident
except ImportError:
    try:
        from thread import get_ident
    except ImportError:
        from _thread import get_ident


class CameraEvent(object):
    """An Event-like class that signals all active clients when a new frame is
    available.
    """
    def __init__(self):
        self.events = {}

    def wait(self):
        """Invoked from each client's thread to wait for the next frame."""
        ident = get_ident()
        if ident not in self.events:
            # this is a new client
            # add an entry for it in the self.events dict
            # each entry has two elements, a threading.Event() and a timestamp
            self.events[ident] = [threading.Event(), time.time()]
        return self.events[ident][0].wait()

    def set(self):
        """Invoked by the camera thread when a new frame is available."""
        now = time.time()
        remove = None
        for ident, event in self.events.items():
            if not event[0].isSet():
                # if this client's event is not set, then set it
                # also update the last set timestamp to now
                event[0].set()
                event[1] = now
            else:
                # if the client's event is already set, it means the client
                # did not process a previous frame
                # if the event stays set for more than 5 seconds, then assume
                # the client is gone and remove it
                if now - event[1] > 5:
                    remove = ident
        if remove:
            del self.events[remove]

    def clear(self):
        """Invoked from each client's thread after a frame was processed."""
        self.events[get_ident()][0].clear()


class BaseCamera(object):
    thread = None  # background thread that reads frames from camera
    frame = None  # current frame is stored here by background thread
    last_access = 0  # time of last client access to the camera
    event = CameraEvent()

    def __init__(self):
        """Start the background camera thread if it isn't running yet."""
        if BaseCamera.thread is None:
            BaseCamera.last_access = time.time()

            # start background frame thread
            BaseCamera.thread = threading.Thread(target=self._thread)
            BaseCamera.thread.start()

            # wait until frames are available
            while self.get_frame() is None:
                time.sleep(0)

    def get_frame(self):
        """Return the current camera frame."""
        BaseCamera.last_access = time.time()

        # wait for a signal from the camera thread
        BaseCamera.event.wait()
        BaseCamera.event.clear()

        return BaseCamera.frame

    @staticmethod
    def frames():
        """"Generator that returns frames from the camera."""
        raise RuntimeError('Must be implemented by subclasses.')

    @classmethod
    def _thread(cls):
        """Camera background thread."""
        print('Starting camera thread.')
        frames_iterator = cls.frames()
        for frame in frames_iterator:
            BaseCamera.frame = frame
            BaseCamera.event.set()  # send signal to clients
            time.sleep(0)

            # if there hasn't been any clients asking for frames in
            # the last 10 seconds then stop the thread
            if time.time() - BaseCamera.last_access > 10:
                frames_iterator.close()
                print('Stopping camera thread due to inactivity.')
                break
        BaseCamera.thread = None

## `camera_opencv.py`

In [3]:
import cv2
#from base_camera import BaseCamera


class Camera(BaseCamera):
    video_source = 0

    @staticmethod
    def set_video_source(source):
        Camera.video_source = source

    @staticmethod
    def frames():
        camera = cv2.VideoCapture(Camera.video_source)
        if not camera.isOpened():
            raise RuntimeError('Could not start camera.')

        while True:
            # read current frame
            _, img = camera.read()

            # encode as a jpeg image and return it
            yield cv2.imencode('.jpg', img)[1].tobytes()

## `camera_pi.py`

This cell should be disabled when the Jupyter isn't running on a Raspberry Pi.

>**NOTE:** Using Jupyter notebook you can click on a cell, press `esc` and then `r`.
That converts it to a "raw" cell.
You can use `esc` + `y` to convert it back a operating cell.

## `camera.py`

In [4]:
import time
#from base_camera import BaseCamera

base = '/home/jeff/Jupyter-Notebooks/src/flask-video-streaming/'

class Camera(BaseCamera):
    """An emulated camera implementation that streams a repeated sequence of
    files 1.jpg, 2.jpg and 3.jpg at a rate of one frame per second."""
    imgs = [open(base + f + '.jpg', 'rb').read() for f in ['1', '2', '3']]

    @staticmethod
    def frames():
        while True:
            time.sleep(1)
            yield Camera.imgs[int(time.time()) % 3]

## `app.py`

In [5]:
#!/usr/bin/env python

# Usage:
#    to start an OpenCV session, do this: CAMERA=opencv python app.py
#    to start an PiCamera session, do this: CAMERA=picamera python app.py


from importlib import import_module
import base64


# import camera driver
#if os.environ.get('CAMERA'):
#    Camera = import_module('camera_' + os.environ['CAMERA']).Camera
#else:
#    from camera import Camera

# Raspberry Pi camera module (requires picamera package)
# from camera_pi import Camera

def send_a_frame(cap):
    frame = cap.read()[1]
    cnt = cv2.imencode('.png',frame)[1]
    b64 = base64.encodestring(cnt)
    html = "<html><img src='data:image/png;base64,"+b64 +"'></html"
    send(html)

if __name__ == '__main__':
    # open video file
    capture = cv2.VideoCapture("/home/jeff/Videos/People-Walking-Shot-From-Above.mp4")
    
    # Check if camera opened successfully
    if (capture.isOpened() is False):
        print("Error opening video stream or file")
        exit
    
    while(capture.isOpened()):
        send_a_frame(capture)
        
        # pre-set ESC or 'q' to exit
        k = cv2.waitKey(30) & 0xFF
        if k == 27 or k == ord('q'):
            break
    
    cv2.destroyAllWindows()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: encodestring() is a deprecated alias, use encodebytes()


TypeError: expected 1-D data, not 2-D data from ndarray

## Sources
* [Flask Video Streaming Revisited](https://blog.miguelgrinberg.com/post/flask-video-streaming-revisited)
* [An introduction to the Flask Python web app framework](https://opensource.com/article/18/4/flask)
* [Accessing the Raspberry Pi Camera with OpenCV and Python](http://www.pyimagesearch.com/2015/03/30/accessing-the-raspberry-pi-camera-with-opencv-and-python/)